In [1]:
"""
===============================================================
 CLEAN MULTI-AGENT ADVANCED SUBMISSION (Kaggle Agents Playground)
===============================================================
Includes:
  ✔ Sequential Agents
  ✔ Parallel Agents
  ✔ Tools: GoogleSearch, CodeExecution, OpenAPI
  ✔ Session & Memory
  ✔ Context Compaction
  ✔ Observability: Logs + Traces
  ✔ A2A Protocol
  ✔ Deployment Wrapper
  ✔ Notebook-safe execution
===============================================================
"""

import json
import asyncio

# ------------------------------------------------------------
# Try importing kaggle_ai_toolbox; fallback to mocks if missing
# ------------------------------------------------------------
try:
    from kaggle_ai_toolbox.agents import Agent, AgentRunner, Task
    from kaggle_ai_toolbox.tools import GoogleSearchTool, CodeExecutionTool, OpenAPITool
    from kaggle_ai_toolbox.memory import InMemorySessionService, MemoryBank
    from kaggle_ai_toolbox.context import compact_context
    from kaggle_ai_toolbox.protocol import A2AProtocol
    from kaggle_ai_toolbox.logging import Logger

    print("Using kaggle_ai_toolbox (official)")
except Exception:
    print("kaggle_ai_toolbox not found — using safe fallback mocks")

    class Logger:
        @staticmethod
        def log(msg): print("[LOG]", msg)
        @staticmethod
        def trace(msg): print("[TRACE]", msg)

    class Task:
        def __init__(self, prompt): self.prompt = prompt

    class Agent:
        def __init__(self, name, instructions, tools=None, protocol=None):
            self.name = name
            self.instructions = instructions
            self.tools = tools or []
            self.protocol = protocol

        async def run(self, task):
            Logger.trace(f"{self.name} executing")
            return f"Result from {self.name} for: {task.prompt}"

    class GoogleSearchTool:
        async def search(self, q):
            return [f"Result {i} for {q}" for i in range(1, 4)]

    class CodeExecutionTool:
        async def execute(self, code):
            return f"Executed code:\n{code}"

    class OpenAPITool:
        def __init__(self, name, spec, base_url): self.name = name
        async def call(self, path="", method="get", params=None):
            return {"mock": "api_response"}

    class InMemorySessionService: pass
    class MemoryBank: pass
    class A2AProtocol:
        def __init__(self, name): self.name = name
    def compact_context(text): return text[:1200]


# ------------------------------------------------------------
# Tools
# ------------------------------------------------------------

code_tool = CodeExecutionTool()
search_tool = GoogleSearchTool()

weather_api = OpenAPITool(
    name="weather",
    spec="mock-schema",
    base_url="https://mock.weather"
)

pet_api = OpenAPITool(
    name="pet",
    spec="mock-schema",
    base_url="https://mock.petstore"
)


# ------------------------------------------------------------
# Agents
# ------------------------------------------------------------

researcher = Agent(
    name="researcher",
    instructions="Search + summarize information.",
    tools=[search_tool],
)

assistant_a = Agent("assistant_a",
                    "Generate short insights.",
                    tools=[search_tool])

assistant_b = Agent("assistant_b",
                    "Generate additional insights.",
                    tools=[search_tool])

coder = Agent(
    name="coder",
    instructions="Convert insights to Python code.",
    tools=[code_tool],
    protocol=A2AProtocol("researcher→coder"),
)

evaluator = Agent(
    name="evaluator",
    instructions="Score results 1–10.",
)

deployer = Agent(
    name="deploy",
    instructions="Simulate deployment ok message."
)


# ------------------------------------------------------------
# Orchestrated Pipeline
# ------------------------------------------------------------

async def pipeline(user_query: str):
    Logger.log("=== PIPELINE START ===")
    Logger.log("Research phase...")

    # 1. Research
    r = await researcher.run(Task(user_query))

    # 2. Parallel assistants
    Logger.log("Parallel insights...")
    a1, a2 = await asyncio.gather(
        assistant_a.run(Task(user_query)),
        assistant_b.run(Task(user_query))
    )

    # 3. External APIs
    weather = await weather_api.call()
    pet = await pet_api.call()

    # 4. Compact combined context
    combined = f"{r}\n{a1}\n{a2}\n{weather}\n{pet}"
    compacted = compact_context(combined)

    # 5. Generate python code
    Logger.log("Generating Python code...")
    python_code = (
        "def print_insights():\n"
        "    insights = [\n"
        f"        '{r}',\n"
        f"        '{a1}',\n"
        f"        '{a2}',\n"
        f"        '{weather}',\n"
        f"        '{pet}',\n"
        "    ]\n"
        "    for i, it in enumerate(insights, 1): print(f\"{i}. {it}\")\n\n"
        "print_insights()\n"
    )

    code_output = await coder.run(Task(python_code))

    # 6. Evaluate
    score = await evaluator.run(Task(code_output))

    Logger.log("=== PIPELINE END ===")

    return {
        "query": user_query,
        "python_code": python_code,
        "score": score,
        "assistant_outputs": [r, a1, a2],
        "api_data": {"weather": weather, "pet": pet},
    }


# ------------------------------------------------------------
# Deployment Wrapper
# ------------------------------------------------------------

async def deploy_example():
    Logger.log("=== DEPLOY START ===")
    result = await pipeline("Explain Reinforcement Learning with real examples.")
    deploy_msg = await deployer.run(Task("deploy"))
    Logger.log("=== DEPLOY END ===")
    return {**result, "deploy_result": deploy_msg}


# ------------------------------------------------------------
# Notebook-safe execution (NO asyncio.run())
# ------------------------------------------------------------

res = await deploy_example()
print("\n=== FINAL OUTPUT ===")
print(json.dumps(res, indent=2))


kaggle_ai_toolbox not found — using safe fallback mocks
[LOG] === DEPLOY START ===
[LOG] === PIPELINE START ===
[LOG] Research phase...
[TRACE] researcher executing
[LOG] Parallel insights...
[TRACE] assistant_a executing
[TRACE] assistant_b executing
[LOG] Generating Python code...
[TRACE] coder executing
[TRACE] evaluator executing
[LOG] === PIPELINE END ===
[TRACE] deploy executing
[LOG] === DEPLOY END ===

=== FINAL OUTPUT ===
{
  "query": "Explain Reinforcement Learning with real examples.",
  "python_code": "def print_insights():\n    insights = [\n        'Result from researcher for: Explain Reinforcement Learning with real examples.',\n        'Result from assistant_a for: Explain Reinforcement Learning with real examples.',\n        'Result from assistant_b for: Explain Reinforcement Learning with real examples.',\n        '{'mock': 'api_response'}',\n        '{'mock': 'api_response'}',\n    ]\n    for i, it in enumerate(insights, 1): print(f\"{i}. {it}\")\n\nprint_insights()\